In [ ]:
!rm -rf project
!git clone https://github.com/karthu0301/stray-codes-trymachinelearning.git project
%cd project

!touch src/__init__.py
!pip install -q -r requirements.txt pyarrow joblib bs4 pandas scikit-learn matplotlib



In [ ]:
from google.colab import files
uploaded = files.upload()
!mkdir -p data/data
!mv reviews.csv data/data/reviews.csv
!ls -lah data/data


In [ ]:
!python -m src.preprocess \
  --input data/data/reviews.csv \
  --output data/processed/reviews_clean.parquet \
  --labelmap_out data/processed/category_label_map.json \
  --drop_empty_clean \
  --drop_dup_exact \
  --verbose_preview

import pandas as pd
df = pd.read_parquet("data/processed/reviews_clean.parquet")
print("Rows:", len(df), "| Cols:", len(df.columns))
df.head(3)

!python -m src.model \
  --parquet data/processed/reviews_clean.parquet \
  --out_dir artifacts/model_test \
  --ambiguous_strategy valid \
  --test_size 0.2

!ls -lah artifacts/model_test

import json, pandas as pd
from IPython.display import Image, display
from pathlib import Path
out = Path("artifacts/model_test")
print(json.load(open(out/"metrics.json")))
display(pd.read_csv(out/"classification_report.csv"))
display(Image(filename=str(out/"confusion_matrix.png")))

In [ ]:
import pandas as pd
from src.features import assign_label

df = pd.read_parquet("data/processed/reviews_clean.parquet")
heur = df.apply(assign_label, axis=1)

amb = df[heur.isna()]
nonamb = df[~heur.isna()]
!python -m src.preprocess \
  --input data/data/reviews.csv \
  --output data/processed/reviews_clean.parquet \
  --labelmap_out data/processed/category_label_map.json \
  --drop_empty_clean \
  --drop_dup_exact \
  --verbose_preview

import pandas as pd
df = pd.read_parquet("data/processed/reviews_clean.parquet")
print("Rows:", len(df), "| Cols:", len(df.columns))
df.head(3)

!python -m src.model \
  --parquet data/processed/reviews_clean.parquet \
  --out_dir artifacts/model_test \
  --ambiguous_strategy valid \
  --test_size 0.2

!ls -lah artifacts/model_test

!python -m src.model \
  --parquet data/processed/reviews_clean.parquet \
  --out_dir artifacts/model_full_llm \
  --ambiguous_strategy llm \
  --test_size 0.2


!ls -lah artifacts/model_full_llm

import json, pandas as pd
from IPython.display import Image, display
from pathlib import Path
out = Path("artifacts/model_full_llm")
print(json.load(open(out/"metrics.json")))
display(pd.read_csv(out/"classification_report.csv"))
display(Image(filename=str(out/"confusion_matrix.png")))
